# Quantum Mechanics: Solving the Schrödinger Equation in Two Dimensions

By Abinash Das

## Overview

This notebook explores solutions to the two-dimensional Schrödinger equation using numerical methods. The Schrödinger equation is a fundamental equation in quantum mechanics that describes how the quantum state of a physical system changes with time. In stationary problems, it can be used to find the energy levels (eigenvalues) and corresponding wave functions (eigenstates) of a particle in a potential.

## Potentials Explored:
- **2D Hydrogen Atom**
- **Inverted Gaussian Well**
- **Circular Well**
- **Simple Harmonic Oscillator**

## Approach:
1. **Meshgrid Creation**: A two-dimensional grid is created to represent the space in which the particle exists.
2. **Potential Definition**: Different potential functions are defined to simulate various quantum systems.
3. **Finite Difference Method**: The Laplacian in the Hamiltonian operator is approximated using the finite difference method.
4. **Hamiltonian Matrix Construction**: The Hamiltonian matrix, which includes the kinetic and potential energy terms, is constructed.
5. **Eigenvalue Problem**: The eigenvalues and eigenvectors of the Hamiltonian are computed, representing the energy levels and wave functions of the system.
6. **Visualization**: The resulting eigenfunctions are plotted to visualize the possible states of the quantum system.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the meshgrid for x and y
N = 100
L = 2
dx = L / N
X, Y = np.meshgrid(np.linspace(-L / 2, L / 2, N, dtype=float),
                   np.linspace(-L / 2, L / 2, N, dtype=float))

# Define the potential function
def potential(X, Y, potential_type):
    if potential_type == '2D-Hydrogen':
        a = 200
        V = -(np.exp(-a * np.sqrt(X ** 2 + Y ** 2))) / np.sqrt(X ** 2 + Y ** 2)
    elif potential_type == 'Inverted Gaussian Well':
        V = (1.0 - np.exp(-0.5*((X/L)**2 +(Y/L)**2)/0.25**2))
    elif potential_type == 'Circular Well':
        V = 200.0*(1.0 - np.heaviside(2.1*np.pi*np.sqrt((X/L)**2 +(Y/L)**2) - 1, 0))
    elif potential_type == 'Simple Harmonic Oscillator':
        V = X**2 + Y**2
    else:
        raise ValueError("Invalid potential type")
    return V

# Finite difference method for 2D Laplacian
def laplacian_2d(N, dx):
    main_diag = -2 * np.ones(N)
    off_diag = np.ones(N - 1)
    laplacian = np.diag(main_diag) + np.diag(off_diag, 1) + np.diag(off_diag, -1)
    return laplacian / (dx ** 2)

# Getting the potential for different cases
potential_types = ['2D-Hydrogen', 'Inverted Gaussian Well', 'Circular Well', 'Simple Harmonic Oscillator']
for potential_type in potential_types:
    V = potential(X, Y, potential_type)

    # Construct the Hamiltonian matrix
    T = -0.5 * laplacian_2d(N, dx)
    T_total = np.kron(np.eye(N), T) + np.kron(T, np.eye(N))
    U = np.diag(V.reshape(N ** 2))
    H = T_total + U

    # Compute the eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(H)
    eigenfunctions = eigenvectors.T.reshape(-1, N, N)

    # Plotting
    for n in range(10):
        plt.figure(figsize=(8, 6))
        plt.contourf(X, Y, eigenfunctions[n], 100)
        plt.colorbar()
        plt.title(f"Eigenfunction for state {n} with {potential_type} potential")
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.savefig(f'Eigenfunction_{potential_type}_state_{n}.png')
        plt.close()


# Normalization of Quantum States

## Significance of Normalization
The wave function describes the quantum state of a particle, and its square magnitude represents the probability density. For the wave function to be physically meaningful, it must be normalized as observables are represnted by square integrable functions. This ensures that the total probability of finding the particle somewhere in space is equal to one.

## How Normalization Works
Normalization involves adjusting the wave function so that the integral (or sum in the case of discrete systems) of its square magnitude over all space equals one. Mathematically, for a wave function $\psi$, this condition is expressed as:

$\int |\psi|^2 \, dV = 1$

In our discrete grid representation, this integral becomes a sum over all grid points.

## Implementation
In the following section we demonstrate how to normalize eigenfunctions obtained from solving the Schrödinger equation numerically. 

- `normalize_eigenfunction`: This function takes an eigenfunction and the grid spacing `dx` as inputs and returns the normalized eigenfunction.

In [ ]:
def normalize_eigenfunction(eigenfunction, dx):
    norm = np.sum(np.abs(eigenfunction) ** 2) * dx ** 2
    return eigenfunction / np.sqrt(norm)

# Example usage
normalized_eigenfunctions = [normalize_eigenfunction(ef, dx) for ef in eigenfunctions]


# Interactive Visualization of Different Eigenstates
The eigenstates, or wave functions, obtained from solving the Schrödinger equation represent the possible states in which a quantum system can be found. Each state is associated with a specific energy level and a unique spatial distribution of the probability density.

## Understanding Eigenstates
- **Eigenstates**: In quantum mechanics, each eigenstate corresponds to a particular energy level of the system. The square of the wave function's amplitude gives the probability density of finding the particle in a particular location.
- **Higher States**: Higher eigenstates (with larger `n`) correspond to higher energy levels.
Using the interactive slider,  we can select an eigenstate that is displayed as a contour plot, showing the distribution of the probability density, to gain insights into how the spatial probability distribution changes with increasing energy levels.


In [ ]:
from ipywidgets import interact

def plot_eigenstate(n):
    plt.figure(figsize=(8, 6))
    plt.contourf(X, Y, eigenfunctions[n], 100)
    plt.colorbar()
    plt.title(f"Eigenfunction for state {n}")
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()

interact(plot_eigenstate, n=(0, len(eigenfunctions) - 1))


# Time Evolution of Quantum States

## The Schrödinger Equation
The time-dependent Schrödinger equation is given by:

$$i \hbar \frac{\partial}{\partial t} \Psi(\mathbf{r}, t) = \hat{H} \Psi(\mathbf{r}, t)$$

where $\Psi(\mathbf{r}, t)$ is the wave function, $\hat{H}$ is the Hamiltonian operator, and $i$ is the imaginary unit. For time-independent potentials, the solution can be expressed in terms of the eigenstates of the Hamiltonian. The time evolution of a state is given by $\Psi(t) = \hat{U}(t) \Psi(0)$, where $\hat{U}(t)$ is the time evolution operator, defined as $e^{-i\hat{H}t/\hbar}$.


In [ ]:
from scipy.linalg import expm

def time_evolution(eigenstates, eigenvalues, t):
    # Construct the time evolution operator
    U_t = expm(-1j * np.diag(eigenvalues) * t)
    return np.dot(U_t, eigenstates)

# Example: Superpose the first two eigenstates
initial_state = eigenfunctions[0] + eigenfunctions[1]
initial_state /= np.linalg.norm(initial_state)  # Normalize

# Time evolution
t = 1  # Time parameter, can vary
evolved_state = time_evolution(initial_state.reshape(-1), eigenvalues, t)
evolved_state = evolved_state.reshape(N, N)

# Plot the evolved state
plt.figure(figsize=(8, 6))
plt.contourf(X, Y, np.abs(evolved_state)**2, 100)
plt.colorbar()
plt.title("Time Evolved State |Ψ(t)|² at t=" + str(t))
plt.xlabel('X')
plt.ylabel('Y')
plt.show()


# Comparison of Numerical and Analytical Solutions for the Simple Harmonic Oscillator


### The Simple Harmonic Oscillator
The SHO is a cornerstone model in quantum mechanics, representing a particle subject to a quadratic potential. It is one of the few quantum systems that allow for exact solutions, providing an excellent benchmark for numerical methods.

The 1D SHO wave function for the \( n \)-th state is given by:

$$\psi_n(x) = \frac{1}{\sqrt{2^n n!}} \cdot \left(\frac{m\omega}{\pi \hbar}\right)^{1/4} \cdot e^{-\frac{m\omega x^2}{2\hbar}} \cdot H_n\left(\sqrt{\frac{m\omega}{\hbar}} x \right)$$

where $H_n$ is the $n$-th Hermite polynomial, $m$ is the mass, $\omega$ is the angular frequency, and $\hbar$ is the reduced Planck's constant. For simplicity, we can set $m = \omega = \hbar = 1$.

The wave function for the 2D SHO in the $n$-th state is then the product of two 1D wave functions:

$$\psi_{n_x, n_y}(x, y) = \psi_{n_x}(x) \cdot \psi_{n_y}(y)$$

In [ ]:
from scipy.special import hermite
from scipy import pi
from numpy import exp, sqrt, power
import numpy as np
import matplotlib.pyplot as plt

# Define the 1D wave function for the nth state of the SHO
def psi_n(n, x):
    """
    Calculate the nth eigenstate of the 1D harmonic oscillator.

    Args:
    n (int): Quantum number for the eigenstate.
    x (ndarray): Array of position values.

    Returns:
    ndarray: The wave function values for the nth eigenstate.
    """
    Hn = hermite(n)
    normalization_factor = (1 / sqrt(2**n * np.math.factorial(n))) * (1 / pi)**0.25
    return normalization_factor * exp(-x**2 / 2) * Hn(x)

# Define the wave function for the 2D SHO
def analytical_SHO(n, x, y):
    """
    Calculate the product of two 1D SHO wave functions to get the 2D SHO wave function.

    Args:
    n (int): Quantum number for the eigenstate.
    x (ndarray): Array of x position values.
    y (ndarray): Array of y position values.

    Returns:
    ndarray: The wave function values for the nth eigenstate in 2D.
    """
    # The 2D wave function is the outer product of two 1D wave functions
    return np.outer(psi_n(n, x), psi_n(n, y))

# Choose a state for comparison
n = 0  # Ground state, for example
numerical_solution = eigenfunctions[n]
analytical_solution = analytical_SHO(n, X[0], Y[:, 0])

# Plotting both solutions for comparison
plt.figure(figsize=(16, 6))

# Numerical solution plot
plt.subplot(1, 2, 1)
plt.contourf(X, Y, np.abs(numerical_solution)**2, 100)
plt.colorbar()
plt.title('Numerical Solution for n = ' + str(n))
plt.xlabel('X')
plt.ylabel('Y')

# Analytical solution plot
plt.subplot(1, 2, 2)
plt.contourf(X, Y, np.abs(analytical_solution)**2, 100)
plt.colorbar()
plt.title('Analytical Solution for n = ' + str(n))
plt.xlabel('X')
plt.ylabel('Y')

plt.show


## Exporting the data

In [ ]:
import pandas as pd

# Export eigenvalues
np.savetxt("eigenvalues.csv", eigenvalues, delimiter=",")

# Export eigenfunctions
for i, ef in enumerate(eigenfunctions):
    np.savetxt(f"eigenfunction_{i}.csv", ef, delimiter=",")

# Optional: Export in a more structured format like a DataFrame
df = pd.DataFrame({'Eigenvalue': eigenvalues})
df.to_csv("eigenvalues_dataframe.csv", index=False)

print("Data exported successfully.")
